# Machine Learning applied to MuJoCo environment – Models Testing
* ### Author: Mowibox (Ousmane THIONGANE)
* ### Date: November 4th, 2024

## Imports

In [19]:
import os
import numpy as np
import pandas as pd
from chardet import detect
from utils import Model, angular_loss
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, r2_score

The script allows us to load all our data files from the data folder, as well as our models, so that we can test the MAE and R² scores metrics on the different datasets.

## I - Loading the models and the data

### 1.1) Loading the models

In [20]:
models_dir = 'models'
MODELS_2R = []
MODELS_3R = []
MODELS_5R = []

if os.path.exists(models_dir) and os.path.isdir(models_dir):
    for filename in os.listdir(models_dir):
        filepath = os.path.join(models_dir, filename)
        if os.path.isfile(filepath) and filename.endswith('.keras'):
            if '2R' in filename:
                print(f"2R Model found : {filename}")
                model = load_model(filepath, 
                                   custom_objects={'angular_loss': angular_loss})
                model = Model(model=model, 
                              name=filename)
                MODELS_2R.append(model)
            elif '3R' in filename:
                print(f"3R Model found : {filename}")
                model = load_model(filepath, 
                                   custom_objects={'angular_loss': angular_loss})
                model = Model(model=model, 
                              name=filename)
                MODELS_3R.append(model)
            elif '5R' in filename:
                print(f"5R Model found : {filename}")
                model = load_model(filepath, 
                                custom_objects={'angular_loss': angular_loss})
                model = Model(model=model, 
                            name=filename)
                MODELS_5R.append(model)
else:
    print(f"The folder '{models_dir}' does not exists!")


print(f"Number of 2R models : {len(MODELS_2R)}")
print(f"Number of 3R models : {len(MODELS_3R)}")
print(f"Number of 5R models : {len(MODELS_5R)}")

5R Model found : NN4_5R_p75_e20_raw.keras
3R Model found : NN2_3R_p10_e20.keras
3R Model found : NN4_3R_p10_e20_raw.keras
2R Model found : NN2_2R_p10_e20.keras
5R Model found : NN2_5R_p80_e20.keras
2R Model found : NN3_2R_p10_e20_raw.keras
5R Model found : NN3_5R_p75_e20_raw.keras
3R Model found : NN3_3R_p10_e20_raw.keras
2R Model found : NN4_2R_p10_e20_raw.keras
3R Model found : NN1_3R_p10_e20.keras
2R Model found : NN1_2R_p10_e20.keras
Number of 2R models : 4
Number of 3R models : 4
Number of 5R models : 3


### 1.2) Loading the data

In [21]:
data_dir = "data"  
FILENAME_2R = [] 
FILENAME_3R = [] 
FILENAME_5R = []

if os.path.exists(data_dir) and os.path.isdir(data_dir):
    
    for file_name in os.listdir(data_dir):
       
            if "2R" in file_name:
                FILENAME_2R.append(file_name)
            elif "3R" in file_name:
                FILENAME_3R.append(file_name)
            elif "5R" in file_name:
                FILENAME_5R.append(file_name)
else:
    print(f"The folder '{data_dir}' does not exist!")

print("2R data files:", FILENAME_2R)
print("3R data files:", FILENAME_3R)
print("5R data files:", FILENAME_5R)

2R data files: ['2R_1000_100.csv', '2R_2184503_100.csv', '2R_12345_100.csv']
3R data files: ['3R_12345_100.csv', '3R_1000_100.csv', '3R_2184503_100.csv']
5R data files: ['5R_2184503_100.csv', '5R_12345_100.csv', '5R_1000_100.csv']


## II - Computing the metrics performance

### 2.1) 2R robot

In [22]:
for model in MODELS_2R:
    for filename in FILENAME_2R:
        model_2R = model.model
        name_2R =  model.get_name()
        path = f'data/{filename}'

        with open(path, 'rb') as f:
            result = detect(f.read(10000))
            encoding = result['encoding']
        df_2R = pd.read_csv(path, sep=";", header=0, encoding=encoding).copy()

        if 'raw' in name_2R:
            X_2R = df_2R[['j0', ' j1']].values
        else: 
            X_2R = df_2R[[' cos(j0)', ' sin(j0)', ' cos(j1)', ' sin(j1)']].values
        
        y_2R = df_2R[[' ee_x', ' ee_y']].values
    
        y_2R_pred = model_2R.predict(X_2R)
        mae_2R = mean_absolute_error(y_2R, y_2R_pred)
        r2_2R = r2_score(y_2R, y_2R_pred)

        print(f"== Metrics for model {name_2R} on file {filename}")
        print(f"== MAE: {mae_2R}")
        print(f"== R2: {r2_2R}\n")

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 624us/step
== Metrics for model NN2_2R_p10_e20.keras on file 2R_1000_100.csv
== MAE: 0.0029924277795683038
== R2: 0.9983439493939505

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 687us/step
== Metrics for model NN2_2R_p10_e20.keras on file 2R_2184503_100.csv
== MAE: 0.002953136384340733
== R2: 0.9983979681968668

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 617us/step
== Metrics for model NN2_2R_p10_e20.keras on file 2R_12345_100.csv
== MAE: 0.002911944969158036
== R2: 0.9983750263677955

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 614us/step
== Metrics for model NN3_2R_p10_e20_raw.keras on file 2R_1000_100.csv
== MAE: 0.005019118853493233
== R2: 0.9948220046136331

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 637us/step
== Metrics for model NN3_2R_p10_e20_raw.keras on file 2R_2184503_100.csv
== MAE: 0.004992057343594744
== R2: 0.9949360684797843

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 664us/step
== Metrics for model NN3_2R_p10_e20_raw.keras on file 2R_12345_100.csv
== MAE: 0.005064557748222086
== R

### 2.2) 3R robot

In [23]:
for model in MODELS_3R:
    for filename in FILENAME_3R:
        model_3R = model.model
        name_3R =  model.get_name()
        path = f'data/{filename}'

        with open(path, 'rb') as f:
            result = detect(f.read(10000))
            encoding = result['encoding']
        df_3R = pd.read_csv(path, sep=";", header=0, encoding=encoding).copy()

        if 'raw' in name_3R:
            X_3R = df_3R[['j0', ' j1', ' j2']].values
        else: 
            X_3R = df_3R[[' cos(j0)', ' sin(j0)', ' cos(j1)', ' sin(j1)', ' cos(j2)', ' sin(j2)']].values
        
        y_3R = df_3R[[' ee_x', ' ee_y']].values
    
        y_3R_pred = model_3R.predict(X_3R)
        mae_3R = mean_absolute_error(y_3R, y_3R_pred)
        r2_3R = r2_score(y_3R, y_3R_pred)

        print(f"== Metrics for model {name_3R} on file {filename}")
        print(f"== MAE: {mae_3R}")
        print(f"== R2: {r2_3R}\n")

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 627us/step
== Metrics for model NN2_3R_p10_e20.keras on file 3R_12345_100.csv
== MAE: 0.0046523127294882186
== R2: 0.9985171691255339

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 626us/step
== Metrics for model NN2_3R_p10_e20.keras on file 3R_1000_100.csv
== MAE: 0.004671905331936109
== R2: 0.9985125604492

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 613us/step
== Metrics for model NN2_3R_p10_e20.keras on file 3R_2184503_100.csv
== MAE: 0.004617201499473006
== R2: 0.9985614205446045

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 708us/step
== Metrics for model NN4_3R_p10_e20_raw.keras on file 3R_12345_100.csv
== MAE: 0.005081614886686964
== R2: 0.9981984742042227

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 697us/step
== Metrics for model NN4_3R_p10_e20_raw.keras on file 3R_1000_100.csv
== MAE: 0.005117475083633148
== R2: 0.9981861605534366

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 712us/step
== Metrics for model NN4_3R_p10_e20_raw.keras on file 3R_2184503_100.csv
== MAE: 0.0051031488739834725
== R2:

### 2.3) 5R robot

In [24]:
for model in MODELS_5R:
    for filename in FILENAME_5R:
        model_5R = model.model
        name_5R =  model.get_name()
        path = f'data/{filename}'

        with open(path, 'rb') as f:
            result = detect(f.read(10000))
            encoding = result['encoding']
        df_5R = pd.read_csv(path, sep=";", header=0, encoding=encoding).copy()

        if 'raw' in name_5R:
            X_5R = df_5R[['j0', ' j1', ' j2', ' j3', ' j4']].values
        else: 
            X_5R = df_5R[[' cos(j0)', ' sin(j0)',
                           ' cos(j1)', ' sin(j1)',
                           ' cos(j2)', ' sin(j2)',
                            ' cos(j3)', ' sin(j3)',
                            ' cos(j4)', ' sin(j4)']].values
        
        y_5R = df_5R[[' ee_x', ' ee_y', ' ee_z']].values
    
        y_5R_pred = model_5R.predict(X_5R)
        mae_5R = mean_absolute_error(y_5R, y_5R_pred)
        r2_5R = r2_score(y_5R, y_5R_pred)

        print(f"== Metrics for model {name_5R} on file {filename}")
        print(f"== MAE: {mae_5R}")
        print(f"== R2: {r2_5R}\n")

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 749us/step
== Metrics for model NN4_5R_p75_e20_raw.keras on file 5R_2184503_100.csv
== MAE: 0.00820825085875925
== R2: 0.9964214998775293

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 774us/step
== Metrics for model NN4_5R_p75_e20_raw.keras on file 5R_12345_100.csv
== MAE: 0.008371550017793538
== R2: 0.9962692111937236

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 772us/step
== Metrics for model NN4_5R_p75_e20_raw.keras on file 5R_1000_100.csv
== MAE: 0.008469788042597544
== R2: 0.9961821359548849

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 697us/step
== Metrics for model NN2_5R_p80_e20.keras on file 5R_2184503_100.csv
== MAE: 0.0046744646903137095
== R2: 0.9988577195182833

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 715us/step
== Metrics for model NN2_5R_p80_e20.keras on file 5R_12345_100.csv
== MAE: 0.004794280347851889
== R2: 0.9987977688663751

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 698us/step
== Metrics for model NN2_5R_p80_e20.keras on file 5R_1000_100.csv
== MAE: 0.004794525141227044
== R2